In [ ]:
# python 3.11
# please run in the console:
# conda create -n bw25 -c cmutel brightway25
# conda activate bw25
# pip install ipykernel
%pip install numpy pandas plotly nbformat

In [ ]:
# for ei3.8
%pip install bw2io==0.9.dev11

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
#import database_explorer as dbex
import bw2analyzer as ba
import bw2data as bd
import bw2calc as bc
import bw2io as bi
import matrix_utils as mu
import bw_processing as bp

first_setup = False
NAME = "ei38-cutoff-25"

if first_setup:
    if NAME in bd.projects:
        bd.projects.delete_project(NAME, True)

bd.projects.set_current(NAME)

if first_setup:
    bi.bw2setup()

if first_setup:
    ei33 = bi.SingleOutputEcospold2Importer('/home/hurtiol/ALFA/datasets/', 'ecoInvent 3.8')
    ei33.apply_strategies()
    ei33.statistics()


In [2]:
if first_setup:
    ei33.write_database(db_name='ecoInvent 3.8')

Not able to determine geocollections for all datasets. This database is not ready for regionalization.
Title: Writing activities to SQLite3 database:
  Started: 02/10/2023 11:15:06
  Finished: 02/10/2023 11:18:25
  Total time elapsed: 00:03:18
  CPU %: 31.50
  Memory %: 0.15
Created database: ecoInvent 3.8


In [2]:
act = bd.Database("ecoInvent 3.8").search("biomethane production, high pressure from synthetic gas, wood, fluidised technology")[0]
#methods = [x for x in bd.methods if 'IPCC' in x[0]]
method = ('IPCC 2013', 'climate change', 'GWP 100a')
lca = bc.LCA({act: 1}, method = method)
lca.lci()
lca.lcia()
lca.score

0.38520281033490816

In [9]:
# modified version of the AssumeDiagonalGraphTraversal.
# includes separate calculation of positive and negative impact
import warnings
from heapq import heappop, heappush
import itertools
from functools import lru_cache

import numpy as np
from scipy import sparse

from bw2calc import spsolve, LCA

class JRCAssumedDiagonalGraphTraversal:
    """
    Traverse a supply chain, following paths of greatest impact.
    This implementation uses a queue of datasets to assess. As the supply chain is traversed, datasets inputs are added to a list sorted by LCA score. Each activity in the sorted list is assessed, and added to the supply chain graph, as long as its impact is above a certain threshold, and the maximum number of calculations has not been exceeded.
    Because the next dataset assessed is chosen by its impact, not its position in the graph, this is neither a breadth-first nor a depth-first search, but rather "importance-first".
    This class is written in a functional style - no variables are stored in *self*, only methods.
    Should be used by calling the ``calculate`` method.
    .. warning:: Graph traversal with multioutput processes only works when other inputs are substituted (see `Multioutput processes in LCA <http://chris.mutel.org/multioutput.html>`__ for a description of multiputput process math in LCA).
    """

    def calculate(self, lca, cutoff=0.005, max_calc=1e5, skip_coproducts=False):
        """
        Traverse the supply chain graph.
        Args:
            * *lca* (dict): An instance of ``bw2calc.lca.LCA``.
            * *cutoff* (float, default=0.005): Cutoff criteria to stop LCA calculations. Relative score of total, i.e. 0.005 will cutoff if a dataset has a score less than 0.5 percent of the total.
            * *max_calc* (int, default=10000): Maximum number of LCA calculations to perform.
        Returns:
            Dictionary of nodes, edges, and number of LCA calculations.
        """
        if not hasattr(lca, "supply_array"):
            lca.lci()
        if not hasattr(lca, "characterized_inventory"):
            lca.lcia()

        supply = lca.supply_array.copy()
        score = lca.score

        if score == 0:
            raise ValueError("Zero total LCA score makes traversal impossible")

        # Create matrix of LCIA CFs times biosphere flows, as these don't
        # change. This is also the unit score of each activity.
        characterized_biosphere = np.array(
            (lca.characterization_matrix * lca.biosphere_matrix).sum(axis=0)
        ).ravel()
        characterized_biosphere_neg = characterized_biosphere.copy()
        characterized_biosphere_neg[characterized_biosphere_neg > 0] = 0
        characterized_biosphere_pos = characterized_biosphere.copy()
        characterized_biosphere_pos[characterized_biosphere_pos < 0] = 0

        heap, nodes, edges = self.initialize_heap(lca, supply, characterized_biosphere, characterized_biosphere_neg, characterized_biosphere_pos)
        nodes, edges, counter = self.traverse(
            heap,
            nodes,
            edges,
            0,
            max_calc,
            cutoff,
            score,
            supply,
            characterized_biosphere,
            characterized_biosphere_neg,
            characterized_biosphere_pos,
            lca,
            skip_coproducts,
        )

        return {
            "nodes": nodes,
            "edges": edges,
            "counter": counter,
        }

    def initialize_heap(self, lca, supply, characterized_biosphere,
            characterized_biosphere_neg,
            characterized_biosphere_pos):
        """
        Create a `priority queue <http://docs.python.org/2/library/heapq.html>`_ or ``heap`` to store inventory datasets, sorted by LCA score.
        Populates the heap with each activity in ``demand``. Initial nodes are the *functional unit*, i.e. the complete demand, and each activity in the *functional unit*. Initial edges are inputs from each activity into the *functional unit*.
        The *functional unit* is an abstract dataset (as it doesn't exist in the matrix), and is assigned the index ``-1``.
        """
        heap, edges = [], []
        nodes = {-1: {"amount": 1, "cum": lca.score, "ind": 1e-6 * lca.score}}
        for index, amount in enumerate(lca.demand_array):
            if amount == 0:
                continue
            cum_score, cum_score_neg, cum_score_pos = self.cumulative_score(
                index, supply, characterized_biosphere,
            characterized_biosphere_neg,
            characterized_biosphere_pos, lca
            )
            heappush(heap, (abs(1 / cum_score), index, str(index)))
            nodes[index] = {
                "amount": float(supply[index]),
                "cum": cum_score,
                "cum_neg": cum_score_neg,
                "cum_pos": cum_score_pos,
                "ind": self.unit_score(index, supply, characterized_biosphere),
            }
            edges.append(
                {
                    "to": -1,
                    "from": str(index),
                    "amount": amount,
                    "exc_amount": amount,
                    "impact": cum_score * amount / float(supply[index]),
                    "impact_neg": cum_score_neg * amount / float(supply[index]),
                    "impact_pos": cum_score_pos * amount / float(supply[index])
                }
            )
        return heap, nodes, edges

    def cumulative_score(self, index, supply, characterized_biosphere,
            characterized_biosphere_neg,
            characterized_biosphere_pos, lca):
        """Compute cumulative LCA score for a given activity"""
        demand = np.zeros((supply.shape[0],))
        demand[index] = (
            supply[index]
            *
            # Normalize by the production amount
            lca.technosphere_matrix[index, index]
        )
        solved_tech = spsolve(lca.technosphere_matrix, demand)
        return (float(
            (characterized_biosphere * solved_tech).sum()
        ),
        float(
            (characterized_biosphere_neg * solved_tech).sum()
        ),
        float(
            (characterized_biosphere_pos * solved_tech).sum()
        ))

    def unit_score(self, index, supply, characterized_biosphere):
        """Compute the LCA impact caused by the direct emissions and resource consumption of a given activity"""
        return float(characterized_biosphere[index] * supply[index])

    def traverse(
        self,
        heap,
        nodes,
        edges,
        counter,
        max_calc,
        cutoff,
        total_score,
        supply,
        characterized_biosphere,
        characterized_biosphere_neg,
        characterized_biosphere_pos,
        lca,
        skip_coproducts,
    ):
        """
        Build a directed graph by traversing the supply chain.
        Node ids are actually technosphere row/col indices, which makes lookup easier.
        Returns:
            (nodes, edges, number of calculations)
        """
        # static_databases = {name for name in databases if databases[name].get("static")}
        # reverse = lca.dicts.activity.reversed

        while heap:
            if counter >= max_calc:
                warnings.warn("Stopping traversal due to calculation count.")
                break
            parent = heappop(heap)
            parent_index = parent[1]
            full_path_parent = parent[2]
            # Skip links from static databases
            # if static_databases and reverse[parent_index][0] in static_databases:
            #     continue

            # Assume that this activity produces its reference product
            scale_value = lca.technosphere_matrix[parent_index, parent_index]
            if scale_value == 0:
                raise ValueError(
                    "Can't rescale activities that produce zero reference product"
                )
            col = lca.technosphere_matrix[:, parent_index].tocoo()
            # Multiply by -1 because technosphere values are negative
            # (consumption of inputs) and rescale
            children = [
                (int(col.row[i]), float(-1 * col.data[i] / scale_value))
                for i in range(col.row.shape[0])
            ]
            for activity, amount in children:
                # Skip values on technosphere diagonal
                if activity == parent_index:
                    continue
                # Skip negative coproducts
                if skip_coproducts and amount <= 0:
                    continue
                counter += 1
                full_path_id = full_path_parent + '-' + str(activity)
                cumulative_score, cum_score_neg, cum_score_pos = self.cumulative_score(
                    activity, supply, characterized_biosphere, characterized_biosphere_neg, characterized_biosphere_pos, lca
                )
                if abs(cumulative_score) < abs(total_score * cutoff):
                    continue

                # flow between activity and parent (Multiply by -1 because technosphere values are negative)
                flow = (
                    -1.0
                    * lca.technosphere_matrix[activity, parent_index]
                    * supply[parent_index]
                )
                total_activity_output = (
                    lca.technosphere_matrix[activity, activity] * supply[activity]
                )

                # Edge format is (to, from, mass amount, cumulative impact)
                edges.append(
                    {
                        "to": full_path_parent,
                        "from": full_path_id,
                        # "full_path_id": full_path_id,
                        # Amount of this link * amount of parent demanding link
                        "amount": flow,
                        # Raw exchange value
                        "exc_amount": amount,
                        # Impact related to this flow
                        "impact": flow / total_activity_output * cumulative_score,
                        "impact_neg": flow / total_activity_output * cum_score_neg,
                        "impact_pos": flow / total_activity_output * cum_score_pos
                    }
                )
                # Want multiple incoming edges, but don't add existing node
                if activity in nodes:
                    continue
                nodes[activity] = {
                    # Total amount of this flow supplied
                    "amount": total_activity_output,
                    # Cumulative score from all flows of this activity
                    "cum": cumulative_score,
                    "cum_neg": cum_score_neg,
                    "cum_pos": cum_score_pos,
                    # Individual score attributable to environmental flows
                    # coming directory from or to this activity
                    "ind": self.unit_score(activity, supply, characterized_biosphere),
                }
                heappush(heap, (abs(1 / cumulative_score), activity, full_path_id))

        return nodes, edges, counter


In [10]:
trav = JRCAssumedDiagonalGraphTraversal().calculate(lca, cutoff=0.05)

(2.596035057818545, 8136, '8136') 8136
(7.655544916458715, 16404, '8136-16404') 16404
(17.45546805194376, 18854, '8136-16404-18854') 18854
(17.569790651477238, 12886, '8136-16404-12886') 12886
(19.423308135297997, 4040, '8136-4040') 4040
(24.217738312751493, 9946, '8136-9946') 9946
(24.843883866002464, 15337, '8136-15337') 15337
(25.0785173885484, 8000, '8136-15337-8000') 8000
(27.434148352416106, 9804, '8136-9804') 9804
(29.83196900958809, 14409, '8136-14409') 14409
(31.36745183096387, 5074, '8136-14409-5074') 5074
(31.30162316721402, 17197, '8136-14409-5074-17197') 17197
(32.32575722624459, 18640, '8136-16404-12886-18640') 18640
(35.791295231408135, 3149, '8136-9946-3149') 3149
(36.457504511353974, 15009, '8136-16404-15009') 15009
(37.83443062473079, 11726, '8136-16404-18854-11726') 11726
(39.560562862503296, 18139, '8136-16404-12886-18139') 18139
(40.81292610188369, 13946, '8136-13946') 13946
(44.526765102501656, 11864, '8136-13946-11864') 11864
(44.59527428834472, 7881, '8136-13946

In [11]:
trav['edges']

[{'to': -1,
  'from': '8136',
  'amount': 1.0,
  'exc_amount': 1.0,
  'impact': 0.3852028103349081,
  'impact_neg': -0.0041985319257974475,
  'impact_pos': 0.3894013422607055},
 {'to': '8136',
  'from': '8136-4040',
  'amount': 0.814000010490703,
  'exc_amount': 0.8140000104904175,
  'impact': 0.05148407252302469,
  'impact_neg': -0.003796729728404199,
  'impact_pos': 0.05528080225142889},
 {'to': '8136',
  'from': '8136-9804',
  'amount': -0.01462200004607952,
  'exc_amount': -0.01462200004607439,
  'impact': 0.03637634803738445,
  'impact_neg': -4.585507161357016e-08,
  'impact_pos': 0.03637639389245605},
 {'to': '8136',
  'from': '8136-9946',
  'amount': 0.008120000362399089,
  'exc_amount': 0.00812000036239624,
  'impact': 0.041284385384511194,
  'impact_neg': -6.6761062231768e-08,
  'impact_pos': 0.041284452145573435},
 {'to': '8136',
  'from': '8136-13946',
  'amount': 0.01462200004607952,
  'exc_amount': 0.01462200004607439,
  'impact': 0.024502041276781197,
  'impact_neg': -0.0

In [16]:
# from Romain to name the activities
id_to_key = {v:k for k, v in lca.activity_dict.items()}
activities = {str(id): bd.get_activity(id_to_key[id]) for id in list(trav["nodes"].keys())[1:]}
activities

{'8136': 'biomethane production, high pressure from synthetic gas, wood, fluidised technology' (cubic meter, CH, None),
 '4040': 'market for wood chips, dry, measured as dry mass' (kilogram, RER, None),
 '9804': 'market for waste mineral oil' (kilogram, CH, None),
 '9946': 'market for zeolite, powder' (kilogram, GLO, None),
 '13946': 'esterification of rape oil' (kilogram, CH, None),
 '14409': 'market for electricity, medium voltage' (kilowatt hour, CH, None),
 '15337': 'market for charcoal' (kilogram, GLO, None),
 '16404': 'market for wood chips, wet, measured as dry mass' (kilogram, CH, None),
 '12886': 'hardwood forestry, mixed species, sustainable forest management' (kilogram, CH, None),
 '15009': 'softwood forestry, mixed species, sustainable forest management' (kilogram, CH, None),
 '18854': 'market for transport, freight, lorry, unspecified' (ton kilometer, RER, None),
 '11250': 'transport, freight, lorry, all sizes, EURO4 to generic market for transport, freight, lorry, unspeci

In [27]:
import plotly.graph_objects as go
import math

ids = [edge["from"] for edge in trav['edges']]
labels = [activities[id.split('-')[-1]]['name']+' ('+activities[id.split('-')[-1]]['location']+')' for id in ids]
labels_short = [label[:18] for label in labels]
parent_ids = [edge["to"] for edge in trav['edges']][1:]
parent_ids = [""] + parent_ids
parents = [activities[id.split('-')[-1]]['name']+' ('+activities[id.split('-')[-1]]['location']+')' if id != '' else '' for id in parent_ids]

data = dict(ids = ids,
            label = labels,            
            #location = [act['location'] for act in activities],
            parent_ids = parent_ids,
            parent = parents,
            labels_short = labels_short,
            value = [math.floor(edge["impact_pos"]*1000) for edge in trav['edges']], #math.floor(edge["impact_pos"]*1100)/1100
            #value_pct = value_pct
        )
# there is an error here, we have to check.
# This is an ugly hack to get all sums of children smaller than parents
data['value'][12] = 10
fig =go.Figure(go.Sunburst(
            #data,
            ids=data['ids'],
            labels=data['labels_short'],
            parents=data['parent_ids'],
            values=data['value'],
            branchvalues="total",
            #color='labels_short',
            #color_continuous_scale='algae',
            hovertext= data['label'],
            #valueformat = '.0f',
            #maxdepth=1
        )
)
df = pd.DataFrame.from_dict(data).set_index('ids')
display(df)


def check_sums(df):
    checker = df.groupby('parent_ids').sum().join(df['value'], rsuffix='_parent')    
    checker['parent'] = checker.apply(lambda row: activities[row.name]['name'] if row.name != '' else '', axis=1)
    print(checker)

#check_sums(df)

fig.show()

,label,parent_ids,parent,labels_short,value
ids,,,,,
8136,"biomethane production, high pressure from synt...",,,biomethane product,389
8136-4040,"market for wood chips, dry, measured as dry ma...",8136,"biomethane production, high pressure from synt...",market for wood ch,55
8136-9804,market for waste mineral oil (CH),8136,"biomethane production, high pressure from synt...",market for waste m,36
8136-9946,"market for zeolite, powder (GLO)",8136,"biomethane production, high pressure from synt...",market for zeolite,41
8136-13946,esterification of rape oil (CH),8136,"biomethane production, high pressure from synt...",esterification of,24
8136-14409,"market for electricity, medium voltage (CH)",8136,"biomethane production, high pressure from synt...",market for electri,32
8136-15337,market for charcoal (GLO),8136,"biomethane production, high pressure from synt...",market for charcoa,40
8136-16404,"market for wood chips, wet, measured as dry ma...",8136,"biomethane production, high pressure from synt...",market for wood ch,130
8136-16404-12886,"hardwood forestry, mixed species, sustainable ...",8136-16404,"market for wood chips, wet, measured as dry ma...","hardwood forestry,",56


In [58]:
ids = [edge["from"] for edge in trav['edges']]
labels = [activities[id]['name']+' ('+activities[id]['location']+')' for id in ids]
labels_short = [label[:18] for label in labels]
parent_ids = [edge["to"] for edge in trav['edges']][1:]
parents = [""] + [activities[id]['name']+' ('+activities[id]['location']+')' for id in parent_ids]

data = dict(label = labels,
            #location = [act['location'] for act in activities],
            parent = parents,
            labels_short = labels_short,
            value = [-math.ceil(edge["impact_neg"]*1000)/1000 for edge in trav['edges']],
            #value_pct = value_pct
        )
fig =go.Figure(go.Sunburst(
            #data,
            ids=data['label'],
            labels=data['labels_short'],
            parents=data['parent'],
            values=data['value'],
            branchvalues="total",
            #color='labels_short',
            #color_continuous_scale='algae',
            hovertext= data['label'],
            #valueformat = '.0f',
        )
)
df = pd.DataFrame.from_dict(data)
display(df)

fig.show()

,label,parent,labels_short,value
0,"biomethane production, high pressure from synt...",,biomethane product,0.004
1,"market for wood chips, dry, measured as dry ma...","biomethane production, high pressure from synt...",market for wood ch,0.003
2,"market for zeolite, powder (GLO)","biomethane production, high pressure from synt...",market for zeolite,0.000
3,market for charcoal (GLO),"biomethane production, high pressure from synt...",market for charcoa,0.000
4,"market for wood chips, wet, measured as dry ma...","biomethane production, high pressure from synt...",market for wood ch,0.000
5,"hardwood forestry, mixed species, sustainable ...","market for wood chips, wet, measured as dry ma...","hardwood forestry,",0.000
6,"market for transport, freight, lorry, unspecif...","market for wood chips, wet, measured as dry ma...",market for transpo,0.000
7,"market for transport, freight, lorry, unspecif...","market for wood chips, dry, measured as dry ma...",market for transpo,0.000
8,charcoal production (GLO),market for charcoal (GLO),charcoal productio,0.000


                                                    value
parent                                                   
                                                    0.004
biomethane production, high pressure from synth...  0.003
market for charcoal (GLO)                           0.000
market for wood chips, dry, measured as dry mas...  0.000
market for wood chips, wet, measured as dry mas...  0.000


/tmp/ipykernel_70182/1205081643.py:31: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [54]:
main_edge = trav['edges'][0]
fig = go.Figure(go.Waterfall(
    orientation = "v",
    measure = ["relative", "relative", "total"],
    x = ['Emissions', 'Absorptions', 'Total'],
    y = [main_edge["impact_pos"], main_edge["impact_neg"], main_edge["impact"]]
    )
)
fig.show()

In [118]:
links = dict(
    source = [edge["from"] for edge in trav['edges']],
    target = [edge["to"] for edge in trav['edges']],
    value = [-edge["impact_neg"] for edge in trav['edges']]
)
fig = go.Figure(data=[go.Sankey(
    link = links
)])

fig.update_layout(title_text="Basic Sankey Diagram", font_size=10)
fig.show()